In [1]:
#importation des librairies python
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from skimage import io
import time

In [2]:
#debut chonometrage
start = time.time()

In [3]:
#importation des donnees
articles = pd.read_csv(r'C:\Users\Stagiaire\Downloads\Timsoft\H&M\articles.csv/articles.csv')
df = pd.read_csv(r'C:\Users\Stagiaire\Downloads\Timsoft\H&M\transactions_train.csv/transactions.csv', chunksize=100000)
users = next(df)

In [4]:
#choix des caracteristiques articles
df = users.merge(articles, on='article_id')
df = df[['t_dat', 'customer_id', 'article_id', 'prod_name', 'product_type_name',
       'product_group_name', 
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name',
       'perceived_colour_master_name',
       'department_name', 'index_name',
       'index_group_name', 'section_name',
       'garment_group_name', 'detail_desc']]
feature_subset = ['product_group_name', 
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name',
       'perceived_colour_master_name',
       'department_name', 'index_name',
       'index_group_name', 'section_name',
       'garment_group_name']

In [5]:
#Choix des caractéristiques pour construire l espace vectoriel
features = feature_subset
df1 = df[['customer_id', 'article_id'] + features]
dummies_df = pd.get_dummies(df1, columns=features)

In [6]:
#selection de la liste des clients ayant effectue au moins deux achats 
minimum_items = 2
groupby_customer = dummies_df.groupby('customer_id')

l = []
cutomer_ids = []
article_ids = []
for key in groupby_customer.groups.keys():
    temp = groupby_customer.get_group(key)
    if temp.article_id.nunique() >= minimum_items:
        l.append(temp.drop('article_id', axis=1).sum(numeric_only=True).values)
        cutomer_ids.append(key)
        article_ids.extend(temp.article_id.values.tolist())

In [7]:
#construction de la matrice Features - User (matrice qui represente l'affinité article - utilisateur)
user_feature = pd.DataFrame(l, columns = dummies_df.columns[2:])
normalized_user_feature = user_feature.div(user_feature.sum(axis=1), axis=0)
normalized_user_feature.insert(0, 'customer_id', cutomer_ids)
normalized_user_feature = normalized_user_feature.set_index('customer_id')

In [8]:
#construction de la matrice Features - Items (articles - caracteristiques)
item_feature = dummies_df.drop_duplicates(subset='article_id')
item_feature = item_feature[item_feature.article_id.isin(article_ids)].drop('customer_id', axis=1)
item_feature = item_feature.set_index('article_id')

In [9]:
#similarite par produit scalaire 
scores = normalized_user_feature.dot(item_feature.T)

In [10]:
#creation du tableau de prediction
df3 = pd.DataFrame( columns =['Client', 'Prediction'])
for i in range(scores.shape[0]):
    cutomer_scores = scores.loc[scores.index[i]]
    customer_id=scores.index[i]
    customer_prev_items = groupby_customer.get_group(customer_id)['article_id']
    prev_dropped = cutomer_scores.drop(customer_prev_items.values)
    ordered = prev_dropped.sort_values(ascending=False).head(10)  
    B=(ordered.index).values
    C=B.tolist()
    df3.loc[i]=[scores.index[i],C]
    
df_reset=df3
df_reset.to_csv("Predictionhm.csv")

In [11]:
#fin du chronometrage 
end = time.time()
print(end - start)

126.17210865020752


In [12]:
#exportation des recommandations
df_reset.to_csv('Predictions.csv')

In [13]:
#construction de la liste des recommandations
def get_rcmnd(customer_id, scores):
    cutomer_scores = scores.loc[customer_id]
    customer_prev_items = groupby_customer.get_group(customer_id)['article_id']
    prev_dropped = cutomer_scores.drop(customer_prev_items.values)
    ordered = prev_dropped.sort_values(ascending=False)  
    return ordered, customer_prev_items

In [14]:
#definition de la fonction d'affichage de la photo du produit à recommander 
def plot_prev(prev_items):
    fig = plt.figure(figsize=(20, 10))
    for item, i in zip(prev_items, range(1, len(prev_items)+1)):
        item = '0' + str(item)
        sub = item[:3]
        image = path + "/"+ sub + "/"+ item +".jpg"
        image = plt.imread(image)
        fig.add_subplot(1, 6, i)
        plt.imshow(image)

In [15]:
#definition de la fonction d'affichage des photos des recommandations
def plot_rcmnd(rcmnds):
    fig = plt.figure(figsize=(20, 10))
    for item, i in zip(rcmnds, range(1, k+1)):
        item = '0' + str(item)
        sub = item[:3]
        image = path + "/"+ sub + "/"+ item +".jpg"
        image = plt.imread(image)
        fig.add_subplot(1, 6, i)
        plt.imshow(image)

In [16]:
#choix de l article a recommander
k = 6
customer_id = scores.index[0]
rcmnds, prev_items = get_rcmnd(customer_id, scores)
rcmnds = rcmnds.index.values[:k]
path = "F:\images"

In [17]:
plot_prev(prev_items)

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\images/064/0640244003.jpg'

<Figure size 1440x720 with 0 Axes>

In [ ]:
plot_rcmnd(rcmnds)